# Apiriori 算法

In [2]:
! pip install mlxtend

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 1.4/1.4 MB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 9.3/9.3 MB 2.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 302.2/302.2 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


## 使用mlxtend库实现

In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

# 加载数据集(简单)
dataset = [["牛奶", "洋葱", "鸡蛋"],
           ["洋葱", "肉", "鸡蛋", "香蕉"],
           ["牛奶", "苹果", "鸡蛋"],
           ["牛奶", "洋葱", "肉", "鸡蛋"],
           ["洋葱","苹果", "鸡蛋"],
           ["鸡蛋", "苹果"],
           ["洋葱", "牛奶", "苹果", "鸡蛋"]]

# 处理数据
# 读取数据集
# data = pd.read_csv('grocery_store.csv', header=None)
# data.groupby([""])
# # 数据预处理
# transactions = []
# for i in range(len(data)):
#     transactions.append([str(data.values[i, j]) for j in range(len(data.columns))])
# 根据同一名用户同一时间作为一行数据

   
# 转换数据集为布尔矩阵
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
print(te_ary)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 使用Apriori算法找出频繁项集 （ 需要输入为dateframe格式）
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)

 # 计算关联规则的置信度和支持度
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7) # return pandas.core.frame.DataFrame | can to excel/csv
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(牛奶),(洋葱),0.571429,0.714286,0.428571,0.750000,1.05,0.020408,1.142857,0.111111
1,(肉),(洋葱),0.285714,0.714286,0.285714,1.000000,1.40,0.081633,inf,0.400000
2,(洋葱),(鸡蛋),0.714286,1.000000,0.714286,1.000000,1.00,0.000000,inf,0.000000
3,(鸡蛋),(洋葱),1.000000,0.714286,0.714286,0.714286,1.00,0.000000,1.000000,0.000000
4,(牛奶),(鸡蛋),0.571429,1.000000,0.571429,1.000000,1.00,0.000000,inf,0.000000
5,(肉),(鸡蛋),0.285714,1.000000,0.285714,1.000000,1.00,0.000000,inf,0.000000
6,(苹果),(鸡蛋),0.571429,1.000000,0.571429,1.000000,1.00,0.000000,inf,0.000000
7,"(洋葱, 牛奶)",(鸡蛋),0.428571,1.000000,0.428571,1.000000,1.00,0.000000,inf,0.000000
8,"(鸡蛋, 牛奶)",(洋葱),0.571429,0.714286,0.428571,0.750000,1.05,0.020408,1.142857,0.111111
9,(牛奶),"(洋葱, 鸡蛋)",0.571429,0.714286,0.428571,0.750000,1.05,0.020408,1.142857,0.111111


## 手撕实现

In [8]:
#-*- coding: utf-8 -*-
from __future__ import print_function
import pandas as pd

#自定义连接函数，用于实现L_{k-1}到C_k的连接
def connect_string(x, ms):
  x = list(map(lambda i:sorted(i.split(ms)), x))
  l = len(x[0])
  r = []
  for i in range(len(x)):
    for j in range(i,len(x)):
      if x[i][:l-1] == x[j][:l-1] and x[i][l-1] != x[j][l-1]:
        r.append(x[i][:l-1]+sorted([x[j][l-1],x[i][l-1]]))
  return r

#寻找关联规则的函数
def find_rule(d, support, confidence, ms = u'--'):
  result = pd.DataFrame(index=['support', 'confidence']) #定义输出结果
  
  support_series = 1.0*d.sum()/len(d) #支持度序列
  column = list(support_series[support_series > support].index) #初步根据支持度筛选
  k = 0
  
  while len(column) > 1:
    k = k+1
    print(u'\n正在进行第%s次搜索...' %k)
    column = connect_string(column, ms)
    print(u'数目：%s...' %len(column))
    sf = lambda i: d[i].prod(axis=1, numeric_only = True) #新一批支持度的计算函数
    
    #创建连接数据，这一步耗时、耗内存最严重。当数据集较大时，可以考虑并行运算优化。
    d_2 = pd.DataFrame(list(map(sf,column)), index = [ms.join(i) for i in column]).T
    
    support_series_2 = 1.0*d_2[[ms.join(i) for i in column]].sum()/len(d) #计算连接后的支持度
    column = list(support_series_2[support_series_2 > support].index) #新一轮支持度筛选
    support_series = support_series.append(support_series_2)
    column2 = []
    
    for i in column: #遍历可能的推理，如{A,B,C}究竟是A+B-->C还是B+C-->A还是C+A-->B？
      i = i.split(ms)
      for j in range(len(i)):
        column2.append(i[:j]+i[j+1:]+i[j:j+1])
    
    cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) #定义置信度序列
 
    for i in column2: #计算置信度序列
      cofidence_series[ms.join(i)] = support_series[ms.join(sorted(i))]/support_series[ms.join(i[:len(i)-1])]
    
    for i in cofidence_series[cofidence_series > confidence].index: #置信度筛选
      result[i] = 0.0
      result[i]['confidence'] = cofidence_series[i]
      result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]
  
  result = result.T.sort_values(['confidence','support'], ascending = False) #结果整理，输出
  print(u'\n结果为：')
  print(result)
  
  return result

In [17]:
# -*- coding: utf-8 -*-
# 使用Apriori算法挖掘菜品订单关联规则

inputfile = '../data/menu_orders.xls'
outputfile = '../tmp/apriori_rules.xls'  # 结果文件
# data = pd.read_excel(inputfile, header=None)
data = [["牛奶", "洋葱", "鸡蛋"],
           ["洋葱", "肉", "鸡蛋", "香蕉"],
           ["牛奶", "苹果", "鸡蛋"],
           ["牛奶", "洋葱", "肉", "鸡蛋"],
           ["洋葱","苹果", "鸡蛋"],
           ["鸡蛋", "苹果"],
           ["洋葱", "牛奶", "苹果", "鸡蛋"]]
data = pd.DataFrame(data)
print(u'\n转换原始数据至0-1矩阵...')
def ct(x): return pd.Series(1, index=x[pd.notnull(x)],dtype = "float64")  # 转换0-1矩阵的过渡函数


b = map(ct, data.to_numpy())  # 用map方式执行
data = pd.DataFrame(list(b)).fillna(0)  # 实现矩阵转换，空值用0填充
print(u'\n转换完毕。')
del b  # 删除中间变量b，节省内存

support = 0.2  # 最小支持度
confidence = 0.5  # 最小置信度
ms = '---'  # 连接符，默认'--'，用来区分不同元素，如A--B。需要保证原始表格中不含有该字符

find_rule(data, support, confidence, ms)#.to_excel(outputfile)  # return dataframe 后续可保存结果


转换原始数据至0-1矩阵...

转换完毕。

正在进行第1次搜索...
数目：10...

正在进行第2次搜索...
数目：7...

正在进行第3次搜索...
数目：0...

结果为：
               support  confidence
洋葱---鸡蛋       0.714286    1.000000
牛奶---鸡蛋       0.571429    1.000000
苹果---鸡蛋       0.571429    1.000000
洋葱---牛奶---鸡蛋  0.428571    1.000000
肉---洋葱        0.285714    1.000000
肉---鸡蛋        0.285714    1.000000
牛奶---苹果---鸡蛋  0.285714    1.000000
肉---鸡蛋---洋葱   0.285714    1.000000
洋葱---肉---鸡蛋   0.285714    1.000000
洋葱---苹果---鸡蛋  0.285714    1.000000
牛奶---洋葱       0.428571    0.750000
牛奶---鸡蛋---洋葱  0.428571    0.750000
鸡蛋---洋葱       0.714286    0.714286
洋葱---牛奶       0.428571    0.600000
洋葱---鸡蛋---牛奶  0.428571    0.600000
鸡蛋---牛奶       0.571429    0.571429
鸡蛋---苹果       0.571429    0.571429


C:\Users\86181\AppData\Local\Temp/ipykernel_18748/554630502.py:44: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) #定义置信度序列
C:\Users\86181\AppData\Local\Temp/ipykernel_18748/554630502.py:44: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) #定义置信度序列
C:\Users\86181\AppData\Local\Temp/ipykernel_18748/554630502.py:44: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) #定义置信度序列


,support,confidence
洋葱---鸡蛋,0.714286,1.000000
牛奶---鸡蛋,0.571429,1.000000
苹果---鸡蛋,0.571429,1.000000
洋葱---牛奶---鸡蛋,0.428571,1.000000
肉---洋葱,0.285714,1.000000
肉---鸡蛋,0.285714,1.000000
牛奶---苹果---鸡蛋,0.285714,1.000000
肉---鸡蛋---洋葱,0.285714,1.000000
洋葱---肉---鸡蛋,0.285714,1.000000
洋葱---苹果---鸡蛋,0.285714,1.000000


# FP-Growth 算法

In [19]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

# 创建示例数据集
dataset = [['Milk', 'Eggs', 'Bread'],
           ['Milk', 'Butter'],
           ['Cheese', 'Bread', 'Butter'],
           ['Milk', 'Eggs', 'Bread', 'Butter'],
           ['Cheese', 'Bread', 'Butter']]

# 使用TransactionEncoder将数据集转换为布尔矩阵
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 使用fpgrowth函数查找频繁项集
frequent_itemsets = fpgrowth(df, min_support=0.2, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.8,(Bread)
1,0.6,(Milk)
2,0.4,(Eggs)
3,0.8,(Butter)
4,0.4,(Cheese)
5,0.6,"(Bread, Butter)"
6,0.4,"(Milk, Bread)"
7,0.4,"(Milk, Butter)"
8,0.2,"(Milk, Bread, Butter)"
9,0.4,"(Milk, Eggs)"
